In [8]:
import requests
import pyspark
from pyspark.sql import SparkSession
from credentials import mysql_username, mysql_password

In [9]:
url = 'https://raw.githubusercontent.com/platformps/LoanDataset/main/loan_data.json'
response = requests.get(url)
data = response.json()
print(f"Status code: {response.status_code}")

spark = SparkSession.builder.appName("API-loan-application").getOrCreate()

# Convert the API response to an Dataframe
df_loan_data = spark.read.json(spark.sparkContext.parallelize([data]))
print("Dataframe succefully created")
df_loan_data.show()


Status code: 200
Dataframe succefully created
+--------------+------------------+--------------+----------+------------+------+------+-------+-------------+-------------+
|Application_ID|Application_Status|Credit_History|Dependents|   Education|Gender|Income|Married|Property_Area|Self_Employed|
+--------------+------------------+--------------+----------+------------+------+------+-------+-------------+-------------+
|      LP001002|                 Y|             1|         0|    Graduate|  Male|medium|     No|        Urban|           No|
|      LP001003|                 N|             1|         1|    Graduate|  Male|medium|    Yes|        Rural|           No|
|      LP001005|                 Y|             1|         0|    Graduate|  Male|   low|    Yes|        Urban|          Yes|
|      LP001006|                 Y|             1|         0|Not Graduate|  Male|   low|    Yes|        Urban|           No|
|      LP001008|                 Y|             1|         0|    Graduate|  Mal

In [3]:
df_loan_data.columns
df_loan_data.printSchema()
df_loan_data.describe().show()
df_loan_data.count()

root
 |-- Application_ID: string (nullable = true)
 |-- Application_Status: string (nullable = true)
 |-- Credit_History: long (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Income: string (nullable = true)
 |-- Married: string (nullable = true)
 |-- Property_Area: string (nullable = true)
 |-- Self_Employed: string (nullable = true)

+-------+--------------+------------------+------------------+------------------+------------+------+------+-------+-------------+-------------+
|summary|Application_ID|Application_Status|    Credit_History|        Dependents|   Education|Gender|Income|Married|Property_Area|Self_Employed|
+-------+--------------+------------------+------------------+------------------+------------+------+------+-------+-------------+-------------+
|  count|           511|               511|               511|               511|         511|   511|   511|    511|          511| 

511

In [16]:
import pandas as pd
import mysql.connector as mysql
from mysql.connector import Error
import seaborn as sns
import matplotlib.pyplot as plt

In [17]:
try:
    conn = mysql.connect(port = '3306', database = 'creditcard_capstone', 
                        user = mysql_username, password = mysql_password)
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
except Error as e:
    print('Error while connecting to MySQL',e)

You're connected to database:  ('creditcard_capstone',)


In [18]:
sql_query = ("SELECT Application_Status, COUNT(*) AS Count "
                "FROM cdw_sapp_loan_application "
                "WHERE Self_employed = 'Yes' "
                "GROUP BY Application_Status")
cursor.execute(sql_query)                #cursor was assigned in the connect_sql() function
result = cursor.fetchall()          #fetches all the results
df_result = pd.DataFrame(result)
df_result.show()

ProgrammingError: 1146 (42S02): Table 'creditcard_capstone.cdw_sapp_loan_application' doesn't exist

In [33]:

# Write the DataFrame into MySQL
df_loan_data.write.format("jdbc") \
  .mode("append") \
  .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
  .option("dbtable", "creditcard_capstone.cdw-sapp_loan_application") \
  .option("user", "root") \
  .option("password", "password") \
  .save()

# Stop the Spark session
# spark.stop()


Py4JJavaError: An error occurred while calling o852.save.
: java.sql.SQLSyntaxErrorException: You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '-sapp_loan_application (`Application_ID` TEXT , `Application_Status` TEXT , `Cre' at line 1
	at com.mysql.cj.jdbc.exceptions.SQLError.createSQLException(SQLError.java:120)
	at com.mysql.cj.jdbc.exceptions.SQLExceptionsMapping.translateException(SQLExceptionsMapping.java:122)
	at com.mysql.cj.jdbc.StatementImpl.executeUpdateInternal(StatementImpl.java:1333)
	at com.mysql.cj.jdbc.StatementImpl.executeLargeUpdate(StatementImpl.java:2106)
	at com.mysql.cj.jdbc.StatementImpl.executeUpdate(StatementImpl.java:1243)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.executeStatement(JdbcUtils.scala:1083)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.createTable(JdbcUtils.scala:913)
	at org.apache.spark.sql.execution.datasources.jdbc